In [1]:
#hide
#all_slow

# Steel Defect Detection

> POC from multiple Severstal Kaggle Competition solutions.

In [2]:
#![Steel Industry](img.jpg)

In [1]:
#hide
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.vision.all import *
from steel_segmentation.all import * 
import cv2
import ipywidgets as widgets
from ipywidgets import interact, interact_manual, interactive, VBox, HBox
from IPython.display import Image

In [3]:
def opt_func(params, **kwargs): return OptimWrapper(params, torch.optim.Adam, **kwargs)
def splitter(m): return convert_params([[m.encoder], [m.decoder], [m.segmentation_head]])

In [8]:
exported_model_url = "https://www.dropbox.com/s/f52j2u4trox0i6o/efficientnet-b2.pkl?dl=1"

In [ ]:
exported_model = download_data(exported_model_url, "export.pkl")

In [15]:
learner = load_learner(exported_model, cpu=True)

In [16]:
imgid_list = [o.name for o in train_pfiles]
imgid_cond = train.ImageId.isin(imgid_list)
df = train.loc[imgid_cond]
imgid_elements = df.ImageId.drop_duplicates().tolist()

In [17]:
#hide_output
style = {'description_width': 'initial'}
imgid_dropdown = widgets.Dropdown(options=imgid_elements, index=0, description="Select an Image:", style=style)
lbl_pred = widgets.Label()
out_pl = widgets.Output()
btn_run = widgets.Button(description='Detect')
btn_run.style.button_color = 'lightgreen'

In [18]:
#hide_output
btn_upload = widgets.FileUpload(multiple=False)

lbl_pred_upload = widgets.Label()
out_pl_upload = widgets.Output()

btn_run_upload = widgets.Button(description='Detect')
btn_run_upload.style.button_color = 'lightgreen'

clear_upload = widgets.Button(description='Clear', button_style='danger')


In [19]:
def get_defects(preds) -> str:
    preds = preds.float()
    
    zero_preds = torch.zeros(tuple(preds.shape)[1:])
    detected_defects = torch.cat([zero_preds.unsqueeze(axis=0), preds])
    idx_defects = detected_defects.argmax(0).unique()
    argmax_defects = list(np.sort(idx_defects.numpy()))
    types_defects = [str(o) for o in argmax_defects][1:]
    n_defects = len(types_defects)
    
    defects_word = "defects" if n_defects!=1 else "defect"
    types_word = "types" if n_defects!=1 else "type"
    if n_defects > 0:
        return f"Predicted: n°{n_defects} {defects_word} of {types_word}: {' '.join(types_defects)}"
    else:
        return f"Predicted: n°0 {defects_word}"

def segment_img(img, out_widget, lbl_widget, groundtruth=None):
    rles, preds, probs = learner.predict(img)
    title = get_defects(preds)
    img_np = np.array(img)
    w,h,_ = img_np.shape
    
    out_widget.clear_output()
    with out_widget: 
        plot_mask_image("Original", img_np, np.zeros((w,h,4)))
        
        if not (groundtruth is None): 
            plot_mask_image("Ground Truth", img_np, groundtruth)
            
        plot_mask_image("Predicted", np.array(img), preds.permute(1,2,0).float().numpy())
        
    lbl_widget.value = title
    
def on_click_detect(change):
    img = PILImage.create(btn_upload.data[-1]) # new release .content.tobytes()
    segment_img(img, out_pl_upload, lbl_pred_upload)
    
def on_click_clear(change):
    btn_upload._counter = 0
    btn_upload.value.clear()
    out_pl_upload.clear_output()
    lbl_pred_upload.value = ""
        
btn_run_upload.on_click(on_click_detect)
clear_upload.on_click(on_click_clear)

In [20]:
upload_label = widgets.Label('Upload an image and detect steel defects with a Deep Learning segmentation model')
upload_box = VBox([upload_label, 
                   HBox([btn_upload, btn_run_upload, clear_upload]),
                   lbl_pred_upload, 
                   out_pl_upload])

In [21]:
def on_change_show(change):
    imageid = imgid_dropdown.value    
    _, mask = make_mask(imageid)
    
    image_np = cv2.imread(str(train_path/imageid))
    img = PILImage.create(image_np)
    
    segment_img(img, out_pl, lbl_pred, groundtruth=mask)
    
btn_run.on_click(on_change_show)
imgid_dropdown.observe(on_change_show, names='value')

In [22]:
multi_choice_label = widgets.Label('Upload an image and detect steel defects with a Deep Learning segmentation model')
multi_choice_box = VBox([HBox([imgid_dropdown, btn_run]), 
                         lbl_pred,
                         out_pl], style=style)

In [23]:
# tab = widgets.Tab()
# tab.children = [upload_box, multi_choice_box]
# tab.set_title(0, 'Upload validation')
# tab.set_title(1, 'Multiple choice validation')
# tab

In [24]:
accordion = widgets.Accordion(children=[multi_choice_box, upload_box])
accordion.set_title(0, 'Compare results')
accordion.set_title(1, 'Upload validation')
accordion

Accordion(children=(VBox(children=(HBox(children=(Dropdown(description='Select an Image:', options=('72aaba8ad…